In [26]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
from utils import next_season, pct_change_rosters, get_last_year_from_season

In [27]:
engine = create_engine("sqlite:///Data/team_database.db")

In [28]:
#Get data from sqlite database
rosters = pd.read_sql("TEAM_SEASON_ROSTERS", engine)
team_info = pd.read_sql("TEAM_IDS", engine)
team_seasons = pd.read_sql("TEAM_SEASONS", engine)
advanced_stats = pd.read_sql("TEAM_SEASON_ADVANCED_DATA", engine)

In [29]:
"""
Each data point is going to include a season, 
a team, 
the team's change in wins between the specified season and the next.
and the percent change in roster personnel between the specified season and the next.
"""
seasons = []
team_ids = []
team_abbrevs = []
change_num_wins = []
change_rosters = []
change_net_ratings = []

In [30]:
#Filter down to seasons after the 2001-02 season. Don't include the current season.
recent_seasons = [szn for szn in team_seasons["YEAR"].tolist() if int(szn[0:4]) > 2001 and int(szn[0:4]) < 2023]

In [31]:
"""
The strategy for the below is to remove teams that are tanking. Right now, I'm not going to add this to the analysis.
Only include roster changes that don't involve tanking.
This is done by only accepting teams that are top 8 seeds before and after. 
Probably not the most reliable technique.
For instance, the Russell Westbrook trade to the Lakers wasn't tanking, but it resulted in the Lakers
going from a playoff team to missing the play-in. But the strategy should suffice for now. 
"""
"""
filtered_team_seasons = team_seasons[team_seasons["CONF_RANK"] >= 8]
filtered_team_seasons = filtered_team_seasons[filtered_team_seasons["YEAR"].isin(recent_seasons)]
"""
filtered_team_seasons = team_seasons[team_seasons["YEAR"].isin(recent_seasons)]

In [32]:
for index, row in filtered_team_seasons.iterrows():
    matches = filtered_team_seasons[filtered_team_seasons["YEAR"] == next_season(row["YEAR"])]
    matches = matches[matches["TEAM_ID"] == row["TEAM_ID"]]
    if len(matches) == 1:
        following_season = matches.iloc[0]
        #Let's try just getting rid of teams that made a terrible trade or tanking.
        if following_season["WINS"] - row["WINS"] > -20:
            seasons.append(row["YEAR"])
            team_ids.append(row["TEAM_ID"])
            team_abbrevs.append(team_info[team_info["ID"] == row["TEAM_ID"]]["ABBREV"].iloc[0])
            change_num_wins.append(following_season["WINS"] - row["WINS"])

            team_rosters = rosters[rosters["TeamID"] == row["TEAM_ID"]]
            initial_roster = team_rosters[team_rosters["SEASON"] == get_last_year_from_season(row["YEAR"])]
            final_roster = team_rosters[team_rosters["SEASON"] == get_last_year_from_season(following_season["YEAR"])]
            change_rosters.append(pct_change_rosters(initial_roster, final_roster))
            
            team_advanced_stats = advanced_stats[advanced_stats["TEAM_ID"] == row["TEAM_ID"]]
            initial_net_rtg = team_advanced_stats[team_advanced_stats["SEASON"] == row["YEAR"]]["E_NET_RATING"].iloc[0]
            final_net_rtg = team_advanced_stats[team_advanced_stats["SEASON"] == following_season["YEAR"]]["E_NET_RATING"].iloc[0]
            change_net_ratings.append(final_net_rtg-initial_net_rtg)
            

In [35]:
out_dict = {
    "SEASON": seasons, 
    "TEAM_ID": team_ids, 
    "TEAM_ABBREV": team_abbrevs, 
    "CHANGE_NUM_WINS": change_num_wins,
    "CHANGE_ROSTERS": change_rosters,
    "CHANGE_NET_RTG": change_net_ratings
}
out = pd.DataFrame(out_dict)

In [36]:
out.to_sql("WIN_NUM_ROSTER_CHANGE_DATA", engine, if_exists="replace", index=False)

573

In [37]:
out.to_csv("Data/WIN_NUM_ROSTER_CHANGE_DATA.csv")